In [1]:
import numpy as np
import ast
import utm

In [2]:
def from_latlon(latitude, longitude, geozone_num, geozone_let):
    """
    convert lat, lon to (x, y) coordinates (in meters)
    :param latitude:  list or np.array (shape: (n,))
    :param longitude: list or np.array (shape: (n,))
    :param geozone_num: geozone integer
    :param geozone_let: geozone letter
    :return: np.array of (x, y) coordinates (shape: (n, 2))
    """
    xy = np.array([
        utm.from_latlon(
            latitude=lat, 
            longitude=lon, 
            force_zone_number=geozone_num,
            force_zone_letter=geozone_let
        )[:2] for lat, lon in zip(latitude, longitude)
    ])
    return xy


def get_geozone(latitude, longitude):
    """
    :param latitude: 
    :param longitude: 
    :return: UTM geozone number and letter
    """
    geozone_num, geozone_let = utm.from_latlon(latitude, longitude)[2:]
    return geozone_num, geozone_let

In [3]:
def get_area_and_distance(
    tool_width : float,
    time : list,
    track_lat : list,
    track_lon : list,
    path_distance : list,
    field_processed : list,
    field_bounds : list
):
    """
    calculate path distance and processed field area
    :param tool_width: width of the tool (in meters)
    :param track_lat: list or np.array (shape: (n,))
    :param track_lon: list or np.array (shape: (n,))
    :param path_distance: list of pre-calculated path distances (shape: (m,))
    :param field_processed: list of pre-calculated processed area (shape: (m,))
    :param field_bounds: list of GPS subfields bounds
    :return: dictionary with calculated processed area and path distance
    """
    # get working UTM zone number and letter
    geozone_num, geozone_let = get_geozone(track_lat[0], track_lon[0])
    
    # define starting index of calculation
    start_from = len(field_processed)
    
    # convert GPS coordinates of the track
    track_xy = from_latlon(track_lat[start_from:], track_lon[start_from:], geozone_num, geozone_let)
    
    # calculate distances and areas
    diffxy = np.copy(track_xy)
    diffxy[1:] -= diffxy[:-1]
    diffxy[0] = 0
    distances = np.sqrt((diffxy**2).sum(axis=1))
    areas = distances * tool_width
    
    # append calculated distances and areas
    for i in range(len(areas)):
        path_distance.append(path_distance[-1] + distances[i])
        field_processed.append(field_processed[-1] + areas[i])
        
    # create dictionary with output
    area_calc = {
        'tool_width': tool_width,
        'time': time,
        'lat': track_lat,
        'long': track_lon,
        'path_distance': path_distance,
        'field_processed': field_processed
    }

    return area_calc

In [4]:
def update_area_calc(area_calc : dict, field_bounds_str : str):
    """
    update area_calc state 
    :param area_calc: dictionary of the field processing state
    :param field_bounds_str: string representation of the bounds
    :return: updated dictionary of the field processing state 
    
    """
    field_bounds = []
    area_calc = get_area_and_distance(
        tool_width=area_calc['tool_width'],
        time=area_calc['time'],
        track_lat=area_calc['lat'],
        track_lon=area_calc['long'],
        path_distance=area_calc['path_distance'],
        field_processed=area_calc['field_processed'],
        field_bounds=field_bounds
    )
    return area_calc

### Test


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('../fields100.csv')
field_bounds_str = df.iloc[0]['geometry_coordinates']
field_bounds_str[:30]

'[[[23.999951022,50.067867025,-'

In [7]:
def get_polygon(string_coords):
    field_bounds = get_bounds_from_string(string_coords)
    zone_num, zone_let = get_geozone(field_bounds[0][0][1], field_bounds[0][0][0])
    subfields_xy = get_subfields_xy(field_bounds, zone_num, zone_let)
    return unary_union([Polygon(subfield).buffer(0) if len(subfield) > 1 else None for subfield in subfields_xy])

In [8]:
# get_polygon(field_bounds_str)

In [9]:
start_point = np.array([23.990487, 50.063754])
diff = np.random.uniform(-0.008000, 0.011000, (1000, 2))/500
diff.min(), diff.max()

(-1.5998663433538817e-05, 2.195221181360242e-05)

In [10]:
path = [start_point]
for dxy in diff:
    path.append(path[-1] + dxy)
path = np.array(path)

In [11]:
lon, lat = path.T

In [12]:
area_calc = {
    'tool_width' : 8,
    'time' : np.ones(900),
    'lat' : lat[:900],
    'long' : lon[:900],
    'path_distance' : [0,0],
    'field_processed' : [0,0]
}

In [13]:
%%time
calc_dict = update_area_calc(area_calc, field_bounds_str)

CPU times: user 80.4 ms, sys: 2.35 ms, total: 82.8 ms
Wall time: 81.3 ms


In [14]:
dif_calc = {
    'tool_width' : 8,
    'time' : np.ones(1000),
    'lat' : lat,
    'long' : lon,
    'path_distance' : calc_dict['path_distance'],
    'field_processed' : calc_dict['field_processed']
}

In [15]:
%%time
diff_calc = update_area_calc(dif_calc, field_bounds_str)

CPU times: user 20 ms, sys: 718 µs, total: 20.7 ms
Wall time: 20.2 ms


In [16]:
area_calc = {
    'tool_width' : 8,
    'time' : np.ones(300),
    'lat' : lat[:300],
    'long' : lon[:300],
    'path_distance' : [0,0],
    'field_processed' : [0,0]
}

In [17]:
%%time
calc_dict = update_area_calc(area_calc, field_bounds_str)

CPU times: user 36.6 ms, sys: 1.51 ms, total: 38.1 ms
Wall time: 36.8 ms


In [18]:
area_calc['path_distance']

[0,
 0,
 0.0,
 2.250641900476313,
 3.919259359164321,
 5.202414163557101,
 6.402026222159064,
 7.824988123122757,
 9.438358371925448,
 10.264460098955478,
 10.926906092174368,
 12.056928297627683,
 13.897130028095468,
 15.859964673951302,
 16.95280444434432,
 18.422802268033617,
 18.700589308050688,
 20.825831317211776,
 22.00052325399966,
 24.23293475879852,
 25.430465834782723,
 26.285472191660464,
 28.078312991502198,
 28.39773106845936,
 29.78701391412514,
 30.97697891491778,
 32.39670370742131,
 33.604620028104,
 35.865929419493334,
 37.30535181985669,
 37.860575027939106,
 38.77788178515907,
 39.697361998392644,
 41.68452679187866,
 42.81270403632274,
 44.252489801379184,
 46.54761490419391,
 48.02650979156203,
 49.21233620479734,
 51.08655568284075,
 52.91041421341789,
 53.87073316379207,
 55.564456754305716,
 57.47355382502221,
 58.55053700680837,
 59.54886619203905,
 60.48915477708864,
 61.643049923106176,
 62.95406061171496,
 63.56534357589482,
 65.76395848518693,
 66.8537370